<a href="https://colab.research.google.com/github/chacha86/pythonai/blob/main/%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80_%EC%A7%91%EA%B0%92_%EC%98%88%EC%B8%A1_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = 'chasaem'
os.environ['KAGGLE_KEY'] = 'dc20323e9468d835d62d9afe27b9a99c'

!kaggle datasets download -d camnugent/california-housing-prices
!unzip '*.zip'

  0% 0.00/400k [00:00<?, ?B/s]
100% 400k/400k [00:00<00:00, 83.8MB/s]
Archive:  california-housing-prices.zip
  inflating: housing.csv             


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

house = pd.read_csv('/content/housing.csv')

In [ ]:
house.info() # total_bedrooms 결측치 존재, 범주컬럼1, 나머지 수치






In [ ]:
house.corr()['median_house_value'].sort_values()

In [15]:
train = house[['median_income']]
target = house['median_house_value']

from sklearn.model_selection import train_test_split

# 학습/테스트 분리
trd, tsd, trt, tst = train_test_split(train, target, random_state=42)

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

scaled_trd = ss.fit_transform(trd)
scaled_tsd = ss.fit_transform(tsd)

# 표준화는 학습데이터만 하는 것.

from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(trd, trt)
lr.score(trd, trt), lr.score(tsd, tst)










(0.47430292180263645, 0.47083837938023365)

In [19]:
house.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [ ]:

## 모델의 성능을 끌어올리는 것이 목표
## 1. 데이터를 더 많이 넣는 것. -> , 다항회귀
## 다중회귀 적용. 모든 특성 사용.
mtrain = house.drop('median_house_value', axis=1)

## 표준화 -> 범주데이터가 있으면 표준화 불가.
# scaled_mtrain = ss.fit_transform(mtrain)

## 범주 데이터, 수치 데이터 분리
mtrain_cate = mtrain[['ocean_proximity']]
mtrain_num = mtrain.drop('ocean_proximity', axis=1)

## 범주데이터 처리
## 회귀에서는 범주데이터는 처리 불가.
## 범주데이터 > 수치데이터 (인코딩)
## 원핫인코딩
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
encoded_mtrain_cate = encoder.fit_transform(mtrain_cate)

col_names = mtrain['ocean_proximity'].unique()
mtrain_cate_1hot = pd.DataFrame(encoded_mtrain_cate.toarray(), columns=col_names)

mtrain_cate_1hot


In [71]:
## 수치데이터 처리
### 결측치 처리
### 1. 특정값으로 대체(0, 평균, 중앙값, 최빈값 등등)
### 2. 해당 행이나 열을 제외
### 3. 특정 범주로 축소
mtrain['total_bedrooms']

# 히스토그램 or 박스플롯

# sns.histplot(data=mtrain, x = 'total_bedrooms')
# sns.boxplot(data=mtrain, y='total_bedrooms')


## 이상치가 많으므로 중앙값으로 대체
mtrain_num['total_bedrooms'] = mtrain_num['total_bedrooms'].fillna(mtrain_num['total_bedrooms'].median())

## 학습/테스트분리하기 전에 범주랑 수치 합치기

mtrain_1hot = pd.concat([mtrain_num, mtrain_cate_1hot], axis=1)


## 학습/테스트분리
mtrd, mtsd, mtrt, mtst = train_test_split(mtrain_1hot, target, random_state=42)

## 표준화
scaled_mtrd = ss.fit_transform(mtrd)
scaled_mtsd = ss.transform(mtsd)

lr2 = LinearRegression()
lr2.fit(scaled_mtrd, mtrt)
lr2.score(scaled_mtrd, mtrt), lr2.score(scaled_mtsd, mtst)


(0.6472032405232842, 0.6374825568355214)

In [ ]:
## 회귀 평가 방법 -> RSquare, RMSE
from sklearn.metrics import mean_squared_error

predict_result = lr2.predict(scaled_mtrd)

#predict_result[:5], mtrt[:5]
mse = mean_squared_error(mtrt, predict_result)

np.sqrt(mse) # 실제값과 예측값 사이에 평균 68000 정도의 차이가 발생한다.

## 분포 -> 히스토그램, 박스플롯

#sns.histplot(x=target)
sns.boxplot(x=target)

## 대부분의 집값 120000 ~ 270000 사이


In [ ]:


#mtrain
#mtrain_cate['ocean_proximity'].value_counts()

In [ ]:

## 2. 데이터 튜닝 - 스케일링, 피처 공학
## 3. 하이퍼파라미터 튜닝 